In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

Duplicate key in file WindowsPath('c:/Users/zoeyz/AppData/Local/Programs/Python/Python310/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 412 ('axes.unicode_minus: True  # use Unicode for the minus symbol rather than hyphen.  See')


In [2]:
from tqdm import tqdm
np.set_printoptions(suppress=True)

In [54]:
pd.set_option('display.max_columns', None)

## combine all data in different years by country

In [3]:
column_names = ['Rank', 'COE_score', 'Farm', 'weight_(lbs.)', 'High_bid', 'Total_value',
       'Company', 'Farmer/Rep.', 'Altitude', 'Country', 'Year', 'City',
       'Region', 'Coffee_characteristics', 'Flavor', 'Acidity', 'Process',
       'Variety', 'Coffee Growing Area', 'Farm_size']

In [288]:
## check the file path
country = ['Bolivia', 'Brazil', 'Burundi', 'Colombia', 'Costa Rica', 'Ecuador', 'El Salvador', 'Ethiopia', 'Guatemala', 'Honduras', 'Indonesia', 'México', 'Nicaragua', 'Perú', 'Rwanda']
country_df = dict()

for country_name in country:

    csv_files = glob.glob('dataset/'+country_name+'/*.csv') 
    log = pd.DataFrame()

    for file in tqdm(csv_files):

        df = pd.read_csv(file)
        df.columns = column_names
        df['Country'] = country_name
        log = pd.concat([log, df], ignore_index=True)
    
    log.to_csv('dataset/'+country_name+'_all.csv')
    country_df[country_name] = log

100%|██████████| 5/5 [00:00<00:00, 192.34it/s]


100%|██████████| 8/8 [00:00<00:00, 215.99it/s]


In [40]:
country_df.keys()

dict_keys(['Bolivia', 'Brazil', 'Burundi', 'Colombia', 'Costa Rica', 'Ecuador', 'El Salvador', 'Ethiopia', 'Guatemala', 'Honduras', 'Indonesia', 'México', 'Nicaragua', 'Perú', 'Rwanda'])

# Preprocessing: Variety

### correcting the names of spelling diversification for building dummy variables

In [ ]:
correcting_dic = {
        'gesha': 'Geisha', 
        'Gesha': 'Geisha',
        'bourbón': 'Bourbon',
        'Bourbón': 'Bourbon',
        'catuaí': 'Catuai',
        'Catuaí': 'Catuai',
        'sl34': 'sl-34',
        'sl 34': 'sl-34',
        'sl28': 'sl-28',
        'sl 28': 'sl-28',
        'yellow bourbon': 'Yellow_Bourbon',
        'red bourbon': 'Red_Bourbon',
        'pink bourbon': 'Pink_Bourbon'}

varieties_wanted = ['geisha', 'pacamara', 'catuai', 'caturra', 'bourbon', 'typica', 'sl-28', 'sl-34', 'colombia', '74112', '74110', '74158']

def variety_dummy_bolivia(df):

    # variety category
    print('Variety category')
    print(df['Variety'].unique())

    # correct spelling
    df['Variety'] = df['Variety'].astype(str)

    for key, value in correcting_dic.items():
        df['Variety'] = df['Variety'].str.replace(key, value)
    
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('á', 'a'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('ã', 'a'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('é', 'e'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('í', 'i'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('ó', 'o'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('ú', 'u'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('ñ', 'n'))

    df['Variety'] = df['Variety'].apply(lambda x: x.replace('Catucai', 'Catuai'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('Cauai', 'Catuai'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('Catui', 'Catuai'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('Cataui', 'Catuai'))

    df['Variety'] = df['Variety'].apply(lambda x: x.replace('Tipica', 'Typica'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('TIPICA', 'Typica'))

    df['Variety'] = df['Variety'].apply(lambda x: x.replace('GESHA', 'Geisha'))

    df['Variety'] = df['Variety'].apply(lambda x: x.replace('Borbon', 'Bourbon'))

    df['Variety'] = df['Variety'].apply(lambda x: x.replace('SL-28', 'sl-28'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('SL28', 'sl-28'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('SL 28', 'sl-28'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('SL-34', 'sl-34'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('SL34', 'sl-34'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('SL 34', 'sl-34'))

    print('Variety category after correcting')
    print(df['Variety'].unique())

    # wanted varieties dummy col
    for v in varieties_wanted:
        df[v] = df['Variety'].apply(lambda x: 1 if v in x or v.capitalize() in x or v.upper() in x else 0)
    
    # mixed_variety dummy col
    df['mixed_variety'] = df['geisha'] + df['pacamara'] + df['catuai'] + df['caturra'] + df['bourbon'] + df['typica'] + df['sl-28'] + df['sl-34'] + df['colombia'] + df['74112'] + df['74110'] + df['74158']
    df['mixed_variety'] = df['mixed_variety'].apply(lambda x: 1 if x>1 else 0)
    
    return df


# Preprocessing: Buyer Locartion

### call Google API for finding the locations of buyers

In [126]:
import requests
import json

In [239]:
api_key = ''

api_id = dict()
api_address = dict()

def call_api(location):

    result_dic = dict()

    if ',' in location:
        location_list = location.split(', ')
        location_list = [i.strip() for i in location_list]
    else:
        location_list = location.split('/ ')
        location_list = [i.strip() for i in location_list]

    for l in location_list:


        headers = {
            'X-Goog-Api-Key': api_key,
            'X-Goog-FieldMask': "places.id,places.formatted_address",
        }

        data = {
            "textQuery": l,
            "maxResultCount": 1,
            "languageCode": "en"
        }

        json.dumps(data)
        response = requests.post('https://places.googleapis.com/v1/places:searchText',
                            data=json.dumps(data),
                            headers=headers)
        result = response.json()

        if len(result) == 0:
            api_id[l] = 'NA'
            api_address[l] = 'NA'
            
            result_dic[l] = result
        
        else:
            api_id[l] = result['places'][0]['id']
            api_address[l] = result['places'][0]['formattedAddress'].split(', ')
        
            result_dic[l] = result
    
    return result_dic

In [222]:
asian_country = ['China', 'India', 'Indonesia', 'Japan', 'Malaysia', 'Philippines', 'Singapore', 'South Korea', 'Taiwan', 'Thailand', 'Vietnam']
north_america_country = ['Canada', 'United States', 'USA']
nordic_country = ['Denmark', 'Finland', 'Iceland', 'Norway', 'Sweden']
europe_country = ['Austria', 'Belgium', 'Czech Republic', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Russia', 'Spain', 'Switzerland', 'Turkey', 'Ukraine', 'United Kingdom', 'UK']

In [283]:
def continent(api_dic):

    coutinent_dic = {'Asia': 0, 'North_America': 0, 'Nordic': 0, 'Europe': 0, 'Others': 0}

    for key, value in api_dic.items():

        if value == {}:
            # print(key)
            coutinent_dic['Others'] = 1
            continue
        
        else:
            address = api_dic[key]['places'][0]['formattedAddress'].split(', ')
            # print(key)
            # print(address)

            if any(item in address for item in asian_country):
                coutinent_dic['Asia'] = 1
            
            elif any(item in address for item in north_america_country):
                coutinent_dic['North_America'] = 1

            elif any(item in address for item in nordic_country):
                coutinent_dic['Nordic'] = 1
            
            elif any(item in address for item in europe_country):
                coutinent_dic['Europe'] = 1
            
            else:
                coutinent_dic['Others'] = 1
    
    return coutinent_dic
    

### bulid the buyer location dummy columns

In [284]:
log['api'] = log['Company'].apply(call_api)
log['continent'] = log['api'].apply(continent)

df_normalized = pd.json_normalize(log['continent'])
log_new = pd.concat([log, df_normalized], axis=1)
log_new

,Rank,Country,Year,COE_score,Farm,High_bid,Total_value,Company,Farmer/Rep.,Altitude,City,Region,Process,Clean_and_clear,Balance_cup,Transparent_cup,Creamy_body,Big_body,Long_aftertaste,Roasted_flavor,Spices_flavor,NuttyCocoa_flavor,Sweet_flavor,Floral_flavor,Fruity_flavor,GreenVegetative_flavor,Winey_flavor,Malic_acidity,Citric_acidity,Tartaric_acidity,Lactic_acidity,Complex_acidity,geisha,pacamara,catuai,caturra,bourbon,typica,sl-28,sl-34,colombia,74112,74110,74158,mixed_variety,api,continent,Asia,North_America,Nordic,Europe,Others
0,1,Bolivia,2004.0,90.44,CENAPROC,11.25,32986.67,Solberg & Hansen AS,CENAPROC (82 farmer lot),1670.0,Caranavi,La Paz,depulped fermented manual wash sun dried on ce...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,{'Solberg & Hansen AS': {'places': [{'id': 'Ch...,"{'Asia': 0, 'North_America': 0, 'Nordic': 1, '...",0,0,1,0,0
1,2,Bolivia,2004.0,90.06,CENAPROC,10.05,27917.14,Maruyama Coffee for Mikatajuku group,Pedro Patana,1650.0,Caranavi,La Paz,depulped fermented manual wash sun dried on ce...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,{'Maruyama Coffee for Mikatajuku group': {'pla...,"{'Asia': 1, 'North_America': 0, 'Nordic': 0, '...",1,0,0,0,0
2,3,Bolivia,2004.0,89.75,CENAPROC,10.00,13889.12,Time’s Club and C-Coop,"Productores Luis Huayhua, Silberio Cereso, Fél...",1700.0,Caranavi,La Paz,depulped fermented manual wash sun dried on wo...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,{'Time’s Club and C-Coop': {}},"{'Asia': 0, 'North_America': 0, 'Nordic': 0, '...",0,0,0,0,1
3,4,Bolivia,2004.0,89.22,Cooperativa UNION PROAGRO,5.25,8101.99,The Roasterie Inc,Eusebio Marca Cruz,1500.0,Caranavi,La Paz,depulped fermented manual wash sun dried on wo...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,{'The Roasterie Inc': {'places': [{'id': 'ChIJ...,"{'Asia': 0, 'North_America': 1, 'Nordic': 0, '...",0,1,0,0,0
4,5,Bolivia,2004.0,88.28,CENAPROC,8.05,11180.74,"Intelligentsia, Coffee Tree Roasters, and The ...","Bernardo Apaza, Lorenzo Choque",1674.0,Caranavi,La Paz,depulped fermented manual wash sun dried on ce...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,{'Intelligentsia': {'places': [{'id': 'ChIJW03...,"{'Asia': 1, 'North_America': 0, 'Nordic': 0, '...",1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2839,24,Rwanda,2018.0,87.24,Gitega Hills Coffee,5.50,11604.01,Wataru for SAZA COFFEE,Mugenzi Emmanuel,2000.0,Nyamagabe,Cyanika,fully washed,0,0,0,0,0,0,0,1,1,2,0,3,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,{'Wataru for SAZA COFFEE': {'places': [{'id': ...,"{'Asia': 1, 'North_America': 0, 'Nordic': 0, '...",1,0,0,0,0
2840,25,Rwanda,2018.0,86.85,Dukundekawa-Abanozaka,5.50,9479.64,Honu Kato Coffee Inc.,Mukahigiro Thacienne,1820.0,Gakenke,Ruli,fully washed,0,0,0,0,0,1,0,0,1,1,1,3,2,0,0,2,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,{'Honu Kato Coffee Inc.': {'places': [{'id': '...,"{'Asia': 1, 'North_America': 0, 'Nordic': 0, '...",1,0,0,0,0
2841,26,Rwanda,2018.0,86.79,Abakundajawa Rushashi,5.50,11343.31,Honu Kato Coffee Inc.,Mukahigiro Thacienne,1762.0,Gakenke,Rushahshi,fully washed,0,0,0,0,0,1,1,1,2,1,0,5,1,0,1,1,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,{'Honu Kato Coffee Inc.': {'places': [{'id': '...,"{'Asia': 1, 'North_America': 0, 'Nordic': 0, '...",1,0,0,0,0
2842,27,Rwanda,2018.0,86.62,Mugeyo CWS,5.50,13260.34,"Wataru & Co., Ltd",Nsengumuremyi Joram,2200.0,Nyamasheke,Kanjongo,fully washed,1,0,0,0,0,0,0,1,1,1,0,8,2,0,0,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,{'Wataru & Co.': {'places': [{'id': 'ChIJAVrCp...,"{'Asia': 1, 'North_America': 0, 'Nordic': 0, '...",1,0,0,0,1


In [290]:
buyer_loc = log_new[['Asia', 'North_America', 'Nordic', 'Europe','Others']]
# buyer_loc.to_csv('C:\\Users\\zoeyz\\Downloads\\buyer_loc.csv')

# Variables: Farm dummy 

In [ ]:
df = pd.read_csv('dataset/all_combined.csv')
flavor_col = ['Clean_and_clear', 'Balance_cup', 'Transparent_cup', 'Creamy_body', 'Big_body', 'Long_aftertaste', 'Roasted_flavor', 'Spices_flavor', 'NuttyCocoa_flavor', 'Sweet_flavor', 'Floral_flavor', 'Fruity_flavor', 'GreenVegetative_flavor', 'Winey_flavor', 'Malic_acidity', 'Citric_acidity', 'Tartaric_acidity', 'Lactic_acidity', 'Complex_acidity']
variety_col = ['geisha', 'pacamara', 'catuai', 'caturra', 'bourbon', 'typica', 'sl-28', 'sl-34', 'colombia', '74112', '74110', '74158', 'mixed_variety']
df_wanted = df[df['Rank', 'COE_score', 'High_bid', 'Altitude'] + flavor_col + variety_col]

In [ ]:
import re
df['Farm'] = df['Farm'].apply(lambda x: x.lower())
df['Farm'] = df['Farm'].apply(lambda x:  re.sub(r"\s?\([^)]*\)", "", x))
df['Farm'] = df['Farm'].apply(lambda x:  re.sub(r"\d+", "", x))
df['Farm'] = df['Farm'].apply(lambda x: x.strip())
df['Farm'].nunique()

1654

### choose the top 100 farms for building farm dummy variables

In [ ]:
c = df['Farm'].value_counts(sort=True).head(100)
c

Farm
la esperanza            19
el cedro                12
buena vista             12
buenos aires            11
los pinos               10
                        ..
la bohemia               3
santa elena              3
mirador – el naranjo     3
el morito ii             3
las brisas               3
Name: count, Length: 100, dtype: int64

In [ ]:
farm_dummy = pd.get_dummies(df, columns=['Farm'], drop_first=False)
farm_dummy.head()

Rank Country  Year  COE_score  High_bid  Total_value  \
0     1  Brazil  2019      92.23      60.1     23850.08   
1     1  Brazil  2019      92.23      50.8     20159.47   
2     2  Brazil  2019      92.15      30.5     10086.35   
3     2  Brazil  2019      92.15      25.1      8300.57   
4     3  Brazil  2019      91.84      12.9      5119.24   

                                             Company  \
0        Maruyama Coffee, Goodboybob Coffee, Harrods   
1                           TAKAMURA COFFEE ROASTERS   
2  \nSAZA COFFEE, Maruyama Coffee, and PROUD MARY...   
3  Time’s Club,  Coffee Libre, Proud Mary, Coffee...   
4                             Wataru for Nozy Coffee   

                   Farmer/Rep.            City              Region  \
0    André Luis Aguila Ribeiro     Ibiraci, MG        Sul de Minas   
1    André Luis Aguila Ribeiro     Ibiraci, MG        Sul de Minas   
2  Leonardo Montesanto Tavares  Angelândia, MG    Chapada de Minas   
3  Leonardo Montesanto Tavares  Angelândia, MG    Chapada de Minas   
4    Merce Jordan Araújo Silva       Piatã, BA  Chapada Diamantina   

          Process  Clean_and_clear  Balance_cup  Transparent_cup  Creamy_body  \
0         natural                0            0                0            0   
1         natural                0            0                0            0   
2  others_process                0            0                0            0   
3  others_process                0            0                0            0   
4  others_process                0            0                0            0   

   Big_body  Long_aftertaste  Roasted_flavor  Spices_flavor  \
0         0                0               0              2   
1         0                0               0              2   
2         0                0               0              0   
3         0                0               0              0   
4         0                0               0              1   

   NuttyCocoa_flavor  Sweet_flavor  Floral_flavor  Fruity_flavor  \
0                  0             3              0              4   
1                  0             3              0              4   
2                  0             1              2              6   
3                  0             1              2              6   
4                  2             6              2              4   

   GreenVegetative_flavor  Winey_flavor  Malic_acidity  Citric_acidity  \
0                       0             0              0               0   
1                       0             0              0               0   
2                       1             0              0               0   
3                       1             0              0               0   
4                       0             0              0               0   

   Tartaric_acidity  Lactic_acidity  Complex_acidity  geisha  pacamara  \
0                 0               0                0       0         0   
1                 0               0                0       0         0   
2                 0               0                0       1         0   
3                 0               0                0       1         0   
4                 0               0                0       0         0   

   catuai  caturra  bourbon  typica  sl-28  sl-34  colombia  74112  74110  \
0       1        0        0       0      0      0         0      0      0   
1       1        0        0       0      0      0         0      0      0   
2       0        0        0       0      0      0         0      0      0   
3       0        0        0       0      0      0         0      0      0   
4       1        0        0       0      0      0         0      0      0   

   74158  mixed_variety  Altitude  Asia  North_America  Nordic  Europe  \
0      0              0    1060.0   0.0            0.0     0.0     0.0   
1      0              0    1060.0   1.0            0.0     0.0     0.0   
2      0              0    1000.0   0.0 

In [ ]:
for farm in df['Farm'].unique():
    if farm not in c.index:
        str = 'Farm_'+farm
        farm_dummy = farm_dummy.drop([str], axis=1)

farm_dummy_df = farm_dummy.iloc[:, -100:].copy().astype('int')
df_wanted = pd.concat([df_wanted, farm_dummy_df], axis=1)

# Variables: Country dummy 

In [ ]:
df = pd.get_dummies(df, columns=['Country'], drop_first=False)
df = df.drop('Country_Indonesia', axis=1)
df[df.columns[-13:]] = df[df.columns[-13:]].astype(int)
con_dummy = df.iloc[:, -13:]
df_wanted = pd.concat([df_wanted, con_dummy], axis=1)

# Variables: Year dummy 

In [ ]:
df = pd.get_dummies(df, columns=['Year'], drop_first=False)
df = df.drop('Year_2011', axis=1)
df[df.columns[-12:]] = df[df.columns[-12:]].astype(int)
year_dummy = df.iloc[:, -12:]
df_wanted = pd.concat([df_wanted, year_dummy], axis=1)

# Variables: Process dummy 

In [ ]:
df = pd.get_dummies(df, columns=['Process'], drop_first=False)
df = df.drop('Process_others_process', axis=1)
df[df.columns[-5:]] = df[df.columns[-5:]].astype(int)
process_dummy = df.iloc[:, -5:]
df_wanted = pd.concat([df_wanted, process_dummy], axis=1)

# Final Dataset

In [ ]:
df_wanted

,Rank,COE_score,High_bid,Clean_and_clear,Balance_cup,Transparent_cup,Creamy_body,Big_body,Long_aftertaste,Roasted_flavor,Spices_flavor,NuttyCocoa_flavor,Sweet_flavor,Floral_flavor,Fruity_flavor,GreenVegetative_flavor,Winey_flavor,Malic_acidity,Citric_acidity,Tartaric_acidity,Lactic_acidity,Complex_acidity,geisha,pacamara,catuai,caturra,bourbon,typica,sl-28,sl-34,colombia,74112,74110,74158,mixed_variety,Altitude,Asia,North_America,Nordic,Europe,Others,Farm_agua dulce,Farm_bella aurora,Farm_bella elizabeth,Farm_bella vista,Farm_betania,Farm_buena vista,Farm_buenos aires,Farm_chiriloma,Farm_divina providencia,Farm_don cayito,Farm_el aguacate,Farm_el bosque,Farm_el cairo,Farm_el cambalache,Farm_el cedro,Farm_el centro,Farm_el cerro,Farm_el conacaste,Farm_el diamante,Farm_el equimite,Farm_el espejo,Farm_el guayabo,Farm_el injerto i,Farm_el matazano,Farm_el mirador,Farm_el morito i,Farm_el nacimiento no.,Farm_el naranjo,Farm_el paraiso,Farm_el paraxaj,Farm_el pino,Farm_el plan,Farm_el porvenir,Farm_el recuerdo,Farm_el roble,Farm_el socorro,Farm_el tambor,Farm_el vergel,Farm_el zapote y anexos,Farm_esperanza,Farm_finca santa cruz,Farm_gakenke,Farm_guatalon,Farm_kalibus la sierra,Farm_la bella,Farm_la bendicion,Farm_la colmena,Farm_la esmeralda,Farm_la españa,Farm_la esperanza,Farm_la fortuna,Farm_la hermosa,Farm_la laguna,Farm_la loma,Farm_la lucuma,Farm_la mina,Farm_la montañita,Farm_la orquidea,Farm_la palma,Farm_la picona,Farm_la planada,Farm_la pradera,Farm_la reforma y anexos,Farm_la salsa,Farm_las brisas,Farm_las brumas,Farm_las delicias,Farm_las duanas,Farm_las flores,Farm_las macadamias,Farm_las nubes,Farm_las palmas,Farm_las ventanas,Farm_las virginias,Farm_liquidambar,Farm_los aguacates,Farm_los morales,Farm_los pinos,Farm_los pirineos,Farm_los robles,Farm_mileydi,Farm_mpanga,Farm_nueva alianza,Farm_nuevo progreso,Farm_ojo de agua,Farm_peña redonda,Farm_plan de la vega,Farm_platanares,Farm_rosma,Farm_san isidro,Farm_san jose,Farm_san luis,Farm_san rafael,Farm_santa elena,Farm_santa lucia,Farm_santa rosa,Farm_santa teresa,Farm_un regalo de dios,Farm_villaure,Farm_yandaro,Country_Brazil,Country_Burundi,Country_Colombia,Country_Costa Rica,Country_Ecuador,Country_El Salvador,Country_Ethiopia,Country_Guatemala,Country_Honduras,Country_México,Country_Nicaragua,Country_Perú,Country_Rwanda,Year_2013,Year_2014,Year_2015,Year_2016,Year_2017,Year_2018,Year_2019,Year_2020,Year_2021,Year_2022,Year_2023,Year_2923,Process_anaerobic,Process_honey,Process_natural,Process_washed,Process_wet hulled
0,1,92.23,60.1,0,0,0,0,0,0,0,2,0,3,0,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1060.000000,0.0,0.0,0.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
1,1,92.23,50.8,0,0,0,0,0,0,0,2,0,3,0,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1060.000000,1.0,0.0,0.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
2,2,92.15,30.5,0,0,0,0,0,0,0,0,0,1,2,6,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1000.000000,0.0,0.0,0.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,2,92.15,25.1,0,0,0,0,0,0,0,0,0,1,2,6,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1000.000000,1.0,0.0,0.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,

In [ ]:
# df_wanted.to_csv('dataset/selected_col_2.csv')